In [1]:
%matplotlib inline
from code1.vis import findPositiveSamples, showCandidate
postiveSample_list = findPositiveSamples()

2023-03-11 19:46:07,667 INFO     pid:7064 code1.dsets:179:__init__ <code1.dsets.LunaDataset object at 0x10412c340>: 551065 training samples


0 CandidateInfoTuple(isNodule_bool=True, diameter_mm=4.653858021, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.584871944187559733312703328980', center_xyz=(71.848635272, 73.8563610762, -83.5882260235))
1 CandidateInfoTuple(isNodule_bool=True, diameter_mm=9.067255434, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.316900421002460665752357657094', center_xyz=(41.9944920605, -3.67968747793, -11.5065805328))
2 CandidateInfoTuple(isNodule_bool=True, diameter_mm=0.0, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.167919147233131417984739058859', center_xyz=(-26.36923194, 53.78148839, -10.106435355))
3 CandidateInfoTuple(isNodule_bool=True, diameter_mm=10.60627713, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.265780642925621389994857727416', center_xyz=(-96.6010989056, -20.9191593806, -120.110927524))
4 CandidateInfoTuple(isNodule_bool=True, diameter_mm=12.13515303, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.219428004988664846407984058588', center_xyz=(122.926512982, 85.6383589837, -213.22

In [2]:
len(postiveSample_list)

100

In [3]:
series_uid = postiveSample_list[11][2]
showCandidate(series_uid)

2023-03-11 19:46:07,711 INFO     pid:7064 code1.dsets:179:__init__ <code1.dsets.LunaDataset object at 0x104143e80>: 522 training samples


<IPython.core.display.Javascript object>

1.3.6.1.4.1.14519.5.2.1.6279.6001.168985655485163461062675655739 175 False [175, 232, 471]


In [4]:
series_uid = postiveSample_list[18][2]
showCandidate(series_uid)

2023-03-11 19:46:08,437 INFO     pid:7064 code1.dsets:179:__init__ <code1.dsets.LunaDataset object at 0x104143e20>: 325 training samples


<IPython.core.display.Javascript object>

1.3.6.1.4.1.14519.5.2.1.6279.6001.282512043257574309474415322775 137 False [137]


In [5]:
# ipyvolume 展示立体图像

In [6]:
import numpy as np
import ipyvolume as ipv
V = np.zeros((128,128,128)) # our 3d array
# outer box
V[30:-30,30:-30,30:-30] = 0.75
V[35:-35,35:-35,35:-35] = 0.0
# inner box
V[50:-50,50:-50,50:-50] = 0.25
V[55:-55,55:-55,55:-55] = 0.0
ipv.quickvolshow(V, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)

/Users/solidus/opt/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/ipyvolume/serialize.py:92: RuntimeWarning: invalid value encountered in divide
  gradient = gradient / np.sqrt(gradient[0] ** 2 + gradient[1] ** 2 + gradient[2] ** 2)
/Users/solidus/opt/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/ipyvolume/serialize.py:102: RuntimeWarning: invalid value encountered in cast
  subdata[..., i] = ((gradient[i][zindex] / 2.0 + 0.5) * 255).astype(np.uint8)


In [7]:
from code1.dsets import getCt
ct = getCt(series_uid)
# 导入 numpy 模块
import numpy as np

# 检查 ct.hu_a 是否是一个 3d 的 numpy 数组
if isinstance(ct.hu_a, np.ndarray) and ct.hu_a.ndim == 3:
    print("ct.hu_a 是一个 3d 的 numpy 数组")
else:
    print("ct.hu_a 不是一个 3d 的 numpy 数组")

# 检查 ct.hu_a 是否有缺失值或异常值
if np.isnan(ct.hu_a).any() or np.isinf(ct.hu_a).any():
    print("ct.hu_a 有缺失值或异常值")
else:
    print("ct.hu_a 没有缺失值或异常值")

# 检查 ct.hu_a 是否在 -1000 到 1000 的范围内
if ct.hu_a.min() >= -1000 and ct.hu_a.max() <= 1000:
    print("ct.hu_a 在 -1000 到 1000 的范围内")
else:
    print("ct.hu_a 不在 -1000 到 1000 的范围内")

ct.hu_a 是一个 3d 的 numpy 数组
ct.hu_a 没有缺失值或异常值
ct.hu_a 在 -1000 到 1000 的范围内


In [8]:
level = [0.25, 0.5, 0.75]
# ipv.quickvolshow(ct.hu_a, level=None, opacity=0.1, level_width=0.1, data_min=-1000, data_max=1000)

In [9]:
import scipy.ndimage.morphology
def build2dLungMask(ct, mask_ndx, threshold_gcc = 0.7):
    dense_mask = ct.hu_a[mask_ndx] > threshold_gcc
    denoise_mask = scipy.ndimage.morphology.binary_closing(dense_mask, iterations=2)
    tissue_mask = scipy.ndimage.morphology.binary_opening(denoise_mask, iterations=10)
    body_mask = scipy.ndimage.morphology.binary_fill_holes(tissue_mask)
    air_mask = scipy.ndimage.morphology.binary_fill_holes(body_mask & ~tissue_mask)

    lung_mask = scipy.ndimage.morphology.binary_dilation(air_mask, iterations=2)

    return air_mask, lung_mask, dense_mask, denoise_mask, tissue_mask, body_mask

def build3dLungMask(ct):
    air_mask, lung_mask, dense_mask, denoise_mask, tissue_mask, body_mask = mask_list = \
        [np.zeros_like(ct.hu_a, dtype=np.bool_) for _ in range(6)]

    for mask_ndx in range(ct.hu_a.shape[0]):
        for i, mask_ary in enumerate(build2dLungMask(ct, mask_ndx)):
            mask_list[i][mask_ndx] = mask_ary

    return air_mask, lung_mask, dense_mask, denoise_mask, tissue_mask, body_mask

In [10]:
ct = getCt(series_uid)
air_mask, lung_mask, dense_mask, denoise_mask, tissue_mask, body_mask = build3dLungMask(ct)

bones = ct.hu_a * (ct.hu_a > 1.5)
lungs = ct.hu_a * air_mask
ipv.figure()
ipv.pylab.volshow(bones + lungs, level=[0.17, 0.17, 0.23], data_min=100, data_max=900)
# ipv.pylab.volshow(bones + lungs, data_min=100, data_max=900)
ipv.show()

/var/folders/n_/54212b493pd645vds1j7659c0000gn/T/ipykernel_7064/1111198128.py:4: DeprecationWarning: Please use `binary_closing` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  denoise_mask = scipy.ndimage.morphology.binary_closing(dense_mask, iterations=2)
/var/folders/n_/54212b493pd645vds1j7659c0000gn/T/ipykernel_7064/1111198128.py:5: DeprecationWarning: Please use `binary_opening` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  tissue_mask = scipy.ndimage.morphology.binary_opening(denoise_mask, iterations=10)
/var/folders/n_/54212b493pd645vds1j7659c0000gn/T/ipykernel_7064/1111198128.py:6: DeprecationWarning: Please use `binary_fill_holes` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  body_mask = scipy.ndimage.morphology.binary_fill_holes(tissue_mask)
/var/folders/n_/54212b493pd645vds1j7659c0000gn/T/ipykernel_7064/1111198128.py:7: DeprecationWarn

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices